# Test04: test code for Broadcast

# Preparation 

## Preparation of google drive (if required)

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [3]:
%cd "ctorch211217/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch211217/MiniCTorch_Prototype/notebook


## Installation of MiniCTorch

In [4]:
! pip install lark-parser

In [5]:
import sys
sys.path.append("../")
import numpy as np
import torch
import minictorch

In [ ]:
! pip install git+https://github.com/kojima-r/MiniCTorch_Prototype.git

## C++コードをコンパイル、実行

In [ ]:
! cat ./test/test04_broadcast.cpp


#include "minictorch.hpp"

void do_broadcast( Tensor &a, Tensor &b )
{
    VariableTensor va(a);
    VariableTensor vb(b);
    
    ListConstructOp  list1;
    list1.set_inputs( &va );
    list1.set_inputs( &vb );
    
    BroadcastTensorsOp  bcast1;
    bcast1.set_inputs( &list1 );
    
    ListUnpackOp unpk0( 0 );
    unpk0.set_inputs( &bcast1 );
    ListUnpackOp unpk1( 1 );
    unpk1.set_inputs( &bcast1 );
    
    AddOp add1;
    add1.set_inputs( &unpk0 );
    add1.set_inputs( &unpk1 );
    add1.set_inputs( NULL );
    
    cout<<"--------------------------------"<<endl;
    cout<<"a"<<a<<endl;
    cout<<"b"<<b<<endl;
    
    list1.forward();
    if( bcast1.forward() )
    {
        unpk0.forward();
        unpk1.forward();
        add1.forward();
        
        cout<<"a_mod"<<unpk0.output<<endl;
        cout<<"b_mod"<<unpk1.output<<endl;
        cout<<"a+b"<<add1.output<<endl;
    }
}

void _broadcast()
{
    Tensor a  = xt::zeros<fprec>({3, 3});
    Tensor b  = xt::arange(3);

In [7]:
C_SRC = './test/test04_broadcast.cpp'
C_OPT = '-I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include '
C_OUT = './test/test04_broadcast'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test04_broadcast.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test04_broadcast


In [8]:
! ./test/test04_broadcast

--------------------------------
a{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b{ 0.,  1.,  2.}
broadcast shape 3,3,
broadcast check=1
a_mod{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b_mod{{ 0.,  1.,  2.},
 { 0.,  1.,  2.},
 { 0.,  1.,  2.}}
a+b{{ 0.,  1.,  2.},
 { 0.,  1.,  2.},
 { 0.,  1.,  2.}}
--------------------------------
a{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b{{ 0.},
 { 1.},
 { 2.}}
broadcast shape 3,3,
broadcast check=1
a_mod{{ 0.,  0.,  0.},
 { 0.,  0.,  0.},
 { 0.,  0.,  0.}}
b_mod{{ 0.,  0.,  0.},
 { 1.,  1.,  1.},
 { 2.,  2.,  2.}}
a+b{{ 0.,  0.,  0.},
 { 1.,  1.,  1.},
 { 2.,  2.,  2.}}
--------------------------------
a{ 0.,  1.,  2.,  3.}
b{{ 0.},
 { 1.},
 { 2.}}
broadcast shape 3,4,
broadcast check=1
a_mod{{ 0.,  1.,  2.,  3.},
 { 0.,  1.,  2.,  3.},
 { 0.,  1.,  2.,  3.}}
b_mod{{ 0.,  0.,  0.,  0.},
 { 1.,  1.,  1.,  1.},
 { 2.,  2.,  2.,  2.}}
a+b{{ 0.,  1.,  2.,  3.},
 { 1.,  2.,  3.,  4.},
 { 2.,  3.,  4.,  5.}}
---------------

## ニューラルネットの定義

In [9]:
class Net(torch.nn.Module):
   def __init__(self, a):
      super(Net, self).__init__()
      self.x = a

   def forward(self,y):
      z = torch.add(self.x, y)
      #print(z)
      return z

## プロジェクト名、作業フォルダ、JSONファイル名の設定

In [10]:
project = "test04"
folder = "./test"
#json_path = folder + '/' + project +'.json'

In [11]:
! mkdir -p test

## 計算グラフをJSONファイルに出力する

In [12]:
def generate_json( json_path, input_x, input_y ):
   model = Net( input_x )
   model.eval()
   with torch.no_grad():
      minictorch.trace( model, input_y, json_path )
   return model

## データ

In [13]:
a  = np.zeros((3, 3),dtype=float)
b  = np.arange(3,dtype=float)
c  = b[:,np.newaxis]
d  = np.arange(4,dtype=float);
e  = [1.0]
g = np.zeros((4, 3), dtype=float );
h = np.arange(6,dtype=float).reshape( (2, 3) );
print("a",a)
print("b",b)
print("c",c)
print("d",d)
print("e",e)
print("g",g)
print("h",h)

a [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
b [0. 1. 2.]
c [[0.]
 [1.]
 [2.]]
d [0. 1. 2. 3.]
e [1.0]
g [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
h [[0. 1. 2.]
 [3. 4. 5.]]


### JSONファイルをC++コードに変換する
### Pythonコードを実行する
### C++コードをコンパイル、実行

## a+b

In [14]:
# a+b
project = 'test04-1'
json_path = folder + '/' + project +'.json'
x = torch.tensor( a )
y = torch.tensor( b , requires_grad=True)
model1 = generate_json( json_path, x, y )

In [15]:
minictorch.convert_all( project, folder, model1, json_path, y, {"inp_data":y}, code="test" )

[JSON] ./test/test04-1.json
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [3, 3], 'constant_value': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'out': [3], 'sorted_id': 0}
[PARAM] ./test/test04-1_param.cpp
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [3, 3], 'constant_value': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'out': [3], 'sorted_id': 0}
{'name': 'input/y', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [3], 'out': [3], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [], 'constant_value': 1.0, 'out': [3], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'aten::add', 'in': [0, 1, 2], 'output_id': 0, 'shape': [3, 3], 'out': [4], 'sorted_id': 3}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [3], 'output_id': 0, 'shape': [3, 3], 'out': [], 'sorted_id': 4}
[CPP]  ./test/test04-1.cpp
[MAKE] ./test/Makefile


In [16]:
# check code
model1.train()
z = model1( y )
print(z)

tensor([[0., 1., 2.],
        [0., 1., 2.],
        [0., 1., 2.]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [17]:
C_SRC = './test/test04-1.cpp ./test/test04-1_param.cpp'
C_OPT = '-I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include'
C_OUT = './test/test04-1'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test04-1.cpp ./test/test04-1_param.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test04-1


In [18]:
! ./test/test04-1

### forward computation ...
{{ 0.,  1.,  2.},
 { 0.,  1.,  2.},
 { 0.,  1.,  2.}}
### backward computation ...
input_grad{{ 1.,  1.,  1.},
 { 1.,  1.,  1.},
 { 1.,  1.,  1.}}


## a+c

In [19]:
# a+c
project = 'test04-2'
json_path = folder + '/' + project +'.json'
x = torch.tensor( a )
y = torch.tensor( c , requires_grad=True)
model2 = generate_json( json_path, x, y )

In [20]:
minictorch.convert_all( project, folder, model2, json_path, y, {"inp_data":y}, code="test" )

[JSON] ./test/test04-2.json
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [3, 3], 'constant_value': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'out': [3], 'sorted_id': 0}
[PARAM] ./test/test04-2_param.cpp
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [3, 3], 'constant_value': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'out': [3], 'sorted_id': 0}
{'name': 'input/y', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [3, 1], 'out': [3], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [], 'constant_value': 1.0, 'out': [3], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'aten::add', 'in': [0, 1, 2], 'output_id': 0, 'shape': [3, 3], 'out': [4], 'sorted_id': 3}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [3], 'output_id': 0, 'shape': [3, 3], 'out': [], 'sorted_id': 4}
[CPP]  ./test/test04-2.cpp
[MAKE] ./test/Makefile


In [21]:
# check code
model2.train()
z = model2( y )
print(z)

tensor([[0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [22]:
C_SRC = './test/test04-2.cpp ./test/test04-2_param.cpp'
C_OUT = './test/test04-2'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test04-2.cpp ./test/test04-2_param.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test04-2


In [23]:
! ./test/test04-2

### forward computation ...
{{ 0.,  0.,  0.},
 { 1.,  1.,  1.},
 { 2.,  2.,  2.}}
### backward computation ...
input_grad{{ 1.,  1.,  1.},
 { 1.,  1.,  1.},
 { 1.,  1.,  1.}}


## d+c

In [24]:
# d+c
project = 'test04-3'
json_path = folder + '/' + project +'.json'
x = torch.tensor( d )
y = torch.tensor( c , requires_grad=True)
model3 = generate_json( json_path, x, y )

In [25]:
minictorch.convert_all( project, folder, model3, json_path, y, {"inp_data":y}, code="test" )

[JSON] ./test/test04-3.json
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [4], 'constant_value': [0.0, 1.0, 2.0, 3.0], 'out': [3], 'sorted_id': 0}
[PARAM] ./test/test04-3_param.cpp
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [4], 'constant_value': [0.0, 1.0, 2.0, 3.0], 'out': [3], 'sorted_id': 0}
{'name': 'input/y', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [3, 1], 'out': [3], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [], 'constant_value': 1.0, 'out': [3], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'aten::add', 'in': [0, 1, 2], 'output_id': 0, 'shape': [3, 4], 'out': [4], 'sorted_id': 3}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [3], 'output_id': 0, 'shape': [3, 4], 'out': [], 'sorted_id': 4}
[CPP]  ./test/test04-3.cpp
[MAKE] ./test/Makefile


In [26]:
# check code
model3.train()
z = model3( y )
print(z)

tensor([[0., 1., 2., 3.],
        [1., 2., 3., 4.],
        [2., 3., 4., 5.]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [27]:
C_SRC = './test/test04-3.cpp ./test/test04-3_param.cpp'
C_OUT = './test/test04-3'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test04-3.cpp ./test/test04-3_param.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test04-3


In [28]:
! ./test/test04-3

### forward computation ...
{{ 0.,  1.,  2.,  3.},
 { 1.,  2.,  3.,  4.},
 { 2.,  3.,  4.,  5.}}
### backward computation ...
input_grad{{ 1.,  1.,  1.,  1.},
 { 1.,  1.,  1.,  1.},
 { 1.,  1.,  1.,  1.}}


## a+e

In [29]:
# a+e
project = 'test04-4'
json_path = folder + '/' + project +'.json'
x = torch.tensor( a )
y = torch.tensor( e , requires_grad=True)
print(e)
print(y)
model4 = generate_json( json_path, x, y )

[1.0]
tensor([1.], requires_grad=True)


In [30]:
minictorch.convert_all( project, folder, model4, json_path, y, {"inp_data":y}, code="test" )

[JSON] ./test/test04-4.json
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [3, 3], 'constant_value': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'out': [3], 'sorted_id': 0}
[PARAM] ./test/test04-4_param.cpp
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [3, 3], 'constant_value': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'out': [3], 'sorted_id': 0}
{'name': 'input/y', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [1], 'out': [3], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [], 'constant_value': 1.0, 'out': [3], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'aten::add', 'in': [0, 1, 2], 'output_id': 0, 'shape': [3, 3], 'out': [4], 'sorted_id': 3}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [3], 'output_id': 0, 'shape': [3, 3], 'out': [], 'sorted_id': 4}
[CPP]  ./test/test04-4.cpp
[MAKE] ./test/Makefile


In [31]:
# check code
model4.train()
z = model4( y )
print(z)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [32]:
C_SRC = './test/test04-4.cpp ./test/test04-4_param.cpp'
C_OUT = './test/test04-4'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test04-4.cpp ./test/test04-4_param.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test04-4


In [33]:
! ./test/test04-4

### forward computation ...
{{ 1.,  1.,  1.},
 { 1.,  1.,  1.},
 { 1.,  1.,  1.}}
### backward computation ...
input_grad{{ 1.,  1.,  1.},
 { 1.,  1.,  1.},
 { 1.,  1.,  1.}}


## g+h  (broadcast error)

In [34]:
# g+h
project = 'test04-5'
json_path = folder + '/' + project +'.json'
x = torch.tensor( g )
y = torch.tensor( h , requires_grad=True)
model5 = generate_json( json_path, x, y )

The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 0
Error occurs, No graph saved


RuntimeError: ignored

In [ ]:
minictorch.convert_all( project, folder, model5, json_path, y, {"inp_data":y}, code="test" )

In [ ]:
# check code
model5.train()
z = model5( y )
print(z)

In [ ]:
C_SRC = './test/test04-5.cpp ./test/test04-5_param.cpp'
C_OUT = './test/test04-5'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

In [ ]:
! ./test/test04-5

## Test Broadcast (python)

In [ ]:
import numpy as np
a = np.zeros((3, 3), np.int)
print(a)
b = np.arange(3)
print(b)

[[0 0 0]
 [0 0 0]
 [0 0 0]]
[0 1 2]


In [ ]:
np.broadcast_to(b,(3,3))

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]])

In [ ]:
c=b.reshape(3,1)
print(c)

[[0]
 [1]
 [2]]


In [ ]:
np.broadcast_to(c,(3,3))

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2]])

In [ ]:
a+b

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]])

In [ ]:
a+c

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2]])

In [ ]:
b+c

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

In [ ]:
d = np.arange(4)
print(d)

[0 1 2 3]


In [ ]:
d+c

array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5]])

In [ ]:
np.broadcast_to(c,(3,4))

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [2, 2, 2, 2]])

In [ ]:
np.broadcast_to(d,(3,4))

array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])

In [ ]:
x = np.zeros((4, 3), dtype=np.int)
print(x)

[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [ ]:
y = np.arange(6).reshape(2, 3)
print(y)

[[0 1 2]
 [3 4 5]]


In [ ]:
x+y

ValueError: ignored

In [ ]:
try:
  print(x+y)
except:
  print("broadcast error. shapes", a.shape, ",",  b.shape)

broadcast error. shapes (3, 3) , (3,)


In [ ]:
def check_broadcast( a, b ):

  import numbers
  def is_scalar(a):
    return isinstance( a, numbers.Number )

  def check_shape( a, n_dim ):
    na = np.ones( n_dim, dtype=np.int )
    inc = n_dim - a.ndim
    for i in range(a.ndim):
      na[i+inc] = a.shape[i]
    return na

  ns = 0
  if is_scalar(a):  ns += 1
  if is_scalar(b):  ns += 1
  if( ns > 0 ): 
    print("scalar =",ns)
    return 2

  n_dim = max( a.ndim, b.ndim )
  na = check_shape( a, n_dim )
  nb = check_shape( b, n_dim )
  print("dimension", n_dim,na,nb)

  ne = 0
  n1 = 0
  n2 = 0
  for i in range(n_dim):
    if na[i] == 1:  
      n1 += 1
    elif nb[i] == 1:  
      n1 += 1
    elif na[i] == nb[i]:
      ne += 1
    else:
      #print("broadcast mismatch dimension no.",i)
      n2 += 1
  print("equal=",ne," eq1 =",n1," other =",n2)
  status = 1;
  if( ne == n_dim ):   
    pass
  elif ( (ne+n1) == n_dim ):  
    pass
  else:
    print("broadcast error. shapes=", a.shape, ",",  b.shape)
    status = 0
  print("status =",status)
  return status

In [ ]:
check_broadcast(a,b)

dimension 2 [3 3] [1 3]
equal= 1  eq1 = 1  other = 0
status = 1


1

In [ ]:
check_broadcast(a,c)

dimension 2 [3 3] [3 1]
equal= 1  eq1 = 1  other = 0
status = 1


1

In [ ]:
check_broadcast(d,c)

dimension 2 [1 4] [3 1]
equal= 0  eq1 = 2  other = 0
status = 1


1

In [ ]:
check_broadcast(x,y)

dimension 2 [4 3] [2 3]
equal= 1  eq1 = 0  other = 1
broadcast error. shapes= (4, 3) , (2, 3)
status = 0


0

In [ ]:
e=1
check_broadcast(a,e)

scalar = 1


2

In [ ]:
a+e

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])